### Find Word Images And Copy

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

file_ext = 1000

# image result parameter
select_image_num = 1  # 10 image sample number big and equal this number
take_image_sample_num = 1000  # 50

In [3]:
word_lemma_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-2-Word In Visual Genome Merge"

word_lemma_data_path2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-3-Find Word Images Mask Fade"

path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-4-Find Word Images And Copy"

#Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_dataframe(df, search_list, target_column, sample_num):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True).head(sample_num)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len()).head(sample_num)               
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_dataframe_all(df, search_list, target_column):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len())
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)        
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [6]:
def take_dataframe_word_sample_from_sorting(df_source, word_list, word_source_column, sort_target_column, sort_ascending=True, sample_num=50):
    '''take_dataframe_word_sample_from_sorting(df_source, word_list, word_source_column, sort_target_column, sort_ascending=True, sample_num=50)\n
    df_source is a dataframe and word_list is equal in word_source_column. Then sort_target_column is sorting according to sort_ascending condition.\n
    Finally, taking sample_num each word_list values.\n 
    ex.\n
    take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat, word_list, "word", "search_text", sort_ascending=True, sample_num=50)
    '''
    df_search_result = pd.DataFrame()
    for word in word_list:
        df_select = df_source[df_source[f"{word_source_column}"] == word]
        df_select = df_select.sort_values(f"{sort_target_column}",key=lambda x:x.str.len(), ascending=sort_ascending).head(sample_num)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [7]:
#def create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, output_path_folder):
#    '''create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, output_path_folder)\n
#    df_source is a dataframe and word_list is equal in word_source_column. Then word image id search in image_id_column and image copy to\n
#    output_path_folder from image_folder_path.\n
#    ex.\n
#    image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"\n
#    output_path_folder = "/home/kurubal/Downloads/temp folder"\n
#    create_word_folder_and_copy_image(df_sample_result, word_list, "word", "image_id", image_path, output_path_folder)
#    '''
#    df_search_result = pd.DataFrame()
#    for word in word_list:
#        path = f"{output_path_folder}/{word}"
#        Path(path).mkdir(parents=True, exist_ok=True)        
#        df_select = df_source[df_source[f"{word_source_column}"] == word]
#        for image_id in df_select[f"{image_id_column}"]:
#            image_file = glob.glob(f"{image_folder_path}/*/{image_id}.jpg")
#            for l in image_file:
#                source = l # source directory
#                destination = path
#                shutil.copy2(source, destination)

In [8]:
def create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, opt_column, image_folder_path, mask_folder_path, output_path_folder):
    '''create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, mask_folder_path, output_path_folder)\n
    df_source is a dataframe and word_list is equal in word_source_column. Then word image id search in image_id_column and image copy to\n
    output_path_folder from mask_folder_path (search in firstly) and image_folder_path. opt_column used for additional features\n
    ex.\n
    image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"\n
    output_path_folder = "/home/kurubal/Downloads/temp folder"\n
    create_word_folder_and_copy_image(df_sample_result, word_list, "word", "image_id", "num", image_path, mask_image_path output_path_folder)
    '''
    df_search_result = pd.DataFrame()
    for word in word_list:
        try:
            path = f"{output_path_folder}/{word}"
            Path(path).mkdir(parents=True, exist_ok=True)        
            df_var = df_source[df_source[f"{word_source_column}"] == word]
            df_select = df_var.drop_duplicates(subset=[f"{image_id_column}"])  # ????
            imageid_list = list(df_select[f"{image_id_column}"])
            opt_column_list = list(df_select[f"{opt_column}"])
            data_list_zip = zip(imageid_list, opt_column_list)
            for image_id, opt in data_list_zip:
                opt_num = int(opt)

                #if isfile(f"{image_folder_path}/VG_Mask_Blur/{image_id}_{opt_num}_mask_blur.jpg"):  # ?????                
                #    image_file = glob.glob(f"{image_folder_path}/*/{image_id}_{opt_num}_mask_blur.jpg")

                if isfile(f"{mask_folder_path}/{image_id}_{opt_num}.jpg"):  # ?????                
                    image_file = glob.glob(f"{mask_folder_path}/{image_id}_{opt_num}.jpg")
                    for l in image_file:
                        source = l # source directory
                        destination = path
                        shutil.copy2(source, destination)
                else:
                    image_file = glob.glob(f"{image_folder_path}/*/{image_id}.jpg")
                    for l in image_file:
                        source = l # source directory
                        destination = path
                        shutil.copy2(source, destination)
        except:
            pass

In [9]:
image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"
mask_image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images/VG_Mask_Blur"

In [10]:
image_folder_list = glob.glob(f"{image_path}/*")
image_folder_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_100K',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_100K_2',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_Mask_Blur']

#### Visual Genome Word Lemma Translate Data

In [156]:
Pos_Tag = "AUX" # NOUN, VERB, ADJ, ADV, NUM, PRON, CCONJ, ADP, AUX

In [157]:
output_path_folder = f"/home/kurubal/Downloads/Image_Select/{Pos_Tag}"

Path(output_path_folder).mkdir(parents=True, exist_ok=True)

In [158]:
df_genome_word_lemma_concat = pd.read_csv(f"{word_lemma_data_path}/Visual_Genome_{file_ext}_Word_Lemma_Search_Result.csv")
df_genome_word_lemma_concat

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,NUM,NaN,bir,bir,bir,a,a,18835735,a,2390994,2920126305
1,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,129190150113
2,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,123317182233
3,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349861,4615142168
4,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349866,54188097
...,...,...,...,...,...,...,...,...,...,...,...
192717,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616,276026737
192718,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604,376517459
192719,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272,95132301236
192720,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210,2211405194


In [159]:
df_genome_word_lemma_concat = df_genome_word_lemma_concat[df_genome_word_lemma_concat["POS1"] == Pos_Tag]
df_genome_word_lemma_concat

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
3187,AUX,NaN,mi,mi,mi,is it,is it,5362714,who is it,2353566,0
3188,AUX,NaN,mi,mi,mi,is it,is it,5362714,who is it,2327286,0
3189,AUX,NaN,mi,mi,mi,is it,is it,5362714,who is it,2369214,0
3190,AUX,NaN,mi,mi,mi,is it,is it,5362714,who is it,2367685,0
3191,AUX,NaN,mi,mi,mi,is it,is it,5362714,how is it,2340925,0
...,...,...,...,...,...,...,...,...,...,...,...
165590,AUX,NaN,miyim,mi,mi,am i,mi,388689,the letters mi on the street sign,2386741,523121973
165591,AUX,NaN,miyim,mi,mi,am i,mi,388689,coburg rd mi min inside right arrow,2366858,30110170180
165592,AUX,NaN,miyim,mi,mi,am i,mi,388689,oakway shopping ctr mi min in right arrow,2366858,40162166213
165593,AUX,NaN,miyim,mi,mi,am i,mi,388689,valley river center mi min straight ahead,2366858,40164166247


In [160]:
df_genome_word_lemma_concat_select = df_genome_word_lemma_concat.groupby("word")["image_id"].apply(lambda x: x.count()>=select_image_num).reset_index()
df_genome_word_lemma_concat_select

,word,image_id
0,ise,True
1,mi,True
2,misin,True
3,misiniz,True
4,miydi,True
5,miyim,True
6,miyiz,True
7,mu,True
8,musun,True
9,musunuz,True


In [161]:
word_select_list = df_genome_word_lemma_concat_select[df_genome_word_lemma_concat_select["image_id"] == True]["word"].to_list()
#word_select_list

In [162]:
df_genome_word_lemma_concat_select_result = df_genome_word_lemma_concat[df_genome_word_lemma_concat["word"].isin(word_select_list)]
df_genome_word_lemma_concat_select_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
3187,AUX,NaN,mi,mi,mi,is it,is it,5362714,who is it,2353566,0
3188,AUX,NaN,mi,mi,mi,is it,is it,5362714,who is it,2327286,0
3189,AUX,NaN,mi,mi,mi,is it,is it,5362714,who is it,2369214,0
3190,AUX,NaN,mi,mi,mi,is it,is it,5362714,who is it,2367685,0
3191,AUX,NaN,mi,mi,mi,is it,is it,5362714,how is it,2340925,0
...,...,...,...,...,...,...,...,...,...,...,...
165590,AUX,NaN,miyim,mi,mi,am i,mi,388689,the letters mi on the street sign,2386741,523121973
165591,AUX,NaN,miyim,mi,mi,am i,mi,388689,coburg rd mi min inside right arrow,2366858,30110170180
165592,AUX,NaN,miyim,mi,mi,am i,mi,388689,oakway shopping ctr mi min in right arrow,2366858,40162166213
165593,AUX,NaN,miyim,mi,mi,am i,mi,388689,valley river center mi min straight ahead,2366858,40164166247


In [163]:
word_list = list(set(df_genome_word_lemma_concat_select_result["word"]))
lemma_list = list(set(df_genome_word_lemma_concat_select_result["lemma.spacy"]))

In [164]:
df_genome_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_select_result, word_list, "word", 
                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
df_genome_sample_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,AUX,NaN,mu,mu,mu,is it,is it,1425461,who is it,2353566,0
1,AUX,NaN,mu,mu,mu,is it,is it,1425461,who is it,2396694,0
2,AUX,NaN,mu,mu,mu,is it,is it,1425461,who is it,2396926,0
3,AUX,NaN,mu,mu,mu,is it,is it,1425461,who is it,2344879,0
4,AUX,NaN,mu,mu,mu,is it,is it,1425461,who is it,2414956,0
...,...,...,...,...,...,...,...,...,...,...,...
1427,AUX,NaN,mü,mü,mü,is it,is it,442313,is it turquoise icing decoration or turquoise ...,2391633,5511023530
1428,AUX,NaN,mü,mü,mü,is it,is it,442313,someone else will know what the pole is for al...,2391989,140136203201
1429,AUX,NaN,mü,mü,mü,is it,is it,442313,it could be a giraffe hard to tell whatever it...,2322521,3552189204
1430,AUX,NaN,mü,mü,mü,is it,is it,442313,because of technique or feigned technique phot...,2369850,4604601819


In [165]:
df_genome_sample_result["word"].nunique()

15

In [166]:
# create folder
# for word in word_list:
#    path = f"{output_path_folder}/ALL_{Pos_Tag}_FOLDER/{word}"
#    Path(path).mkdir(parents=True, exist_ok=True)

In [167]:
# for word
#create_word_folder_and_copy_image(df_genome_sample_result, word_list, "word", "image_id", "num", image_path, mask_image_path ,output_path_folder)

In [168]:
# create folder
for lemma in lemma_list:
    path = f"{output_path_folder}/ALL_{Pos_Tag}_FOLDER/{lemma}"
    Path(path).mkdir(parents=True, exist_ok=True)
    df_var = df_genome_sample_result[df_genome_sample_result["lemma.spacy"] == f"{lemma}"]
    for word in set(df_var["word"]):
        path2 = f"{path}/{word}"
        Path(path2).mkdir(parents=True, exist_ok=True)

In [169]:
# for lemma
create_word_folder_and_copy_image(df_genome_sample_result, lemma_list, "lemma.spacy", "image_id", "num", image_path, mask_image_path ,output_path_folder)

In [ ]:
## Check Result 1
#df_genome_word_lemma_concat_coor = pd.read_csv(f"{word_lemma_data_path2}/Visual_Genome_{file_ext}_Word_Lemma_Coordinate_Search_Result.csv")
#df_genome_word_lemma_concat_coor = df_genome_word_lemma_concat_coor.drop(["height","width","x_koor","y_koor"], axis=1)
#df_genome_word_lemma_concat_coor

In [ ]:
# Check Result 2
#df_genome_word_lemma_concat_select_result_coor_merge = pd.merge(df_genome_word_lemma_concat_select_result,df_genome_word_lemma_concat_coor,how="left", on=["word","image_id","word_en_translate","lemma_en_translate","lemma.spacy","stem","num"])
##df_genome_word_lemma_concat_select_result_coor_merge = pd.merge(df_genome_word_lemma_concat, df_genome_word_lemma_concat_coor,how="left", on=["word","image_id"])
#df_genome_word_lemma_concat_select_result_coor_merge = df_genome_word_lemma_concat_select_result_coor_merge.drop_duplicates()
##df_genome_word_lemma_concat_select_result_coor_merge["num"] = df_genome_word_lemma_concat_select_result_coor_merge["num"].fillna(0)
#df_genome_word_lemma_concat_select_result_coor_merge

In [ ]:
#df_genome_word_lemma_concat_select_result_coor_merge["word"].nunique()

In [ ]:
#word_list = list(set(df_genome_word_lemma_concat_select_result_coor_merge["word"]))
#lemma_list = list(set(df_genome_word_lemma_concat_select_result_coor_merge["lemma.spacy"]))

In [ ]:
#df_genome_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_select_result_coor_merge, word_list, "word", 
#                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
#df_genome_sample_result

In [ ]:
#df_genome_sample_result["word"].nunique()